In [ ]:
from gnomad_db.gnomad_db import gnomAD_DB
import numpy as np
import pandas as pd
import gzip
from tqdm import tqdm
import glob
from joblib import Parallel, delayed

In [ ]:
# initialize database
location = "data"
db = gnomAD_DB(location)

In [ ]:
# read variants from tsv.gz table in batches
def load_batches(file, batch_size=500_000):
    with gzip.open(file, "rb") as f:
        batch = []
        for line in tqdm(f):
            line = line.decode().rstrip()
            if len(batch) == batch_size:
                batch = pd.DataFrame(batch, columns=db.columns).replace(".", np.NaN)
                yield batch
                batch = []

            batch.append(line.split("\t"))
        
        
        if len(batch) != 0:
            batch = pd.DataFrame(batch, columns=db.columns).replace(".", np.NaN)
            yield batch

In [ ]:
tables = glob.glob("data/*.tsv.gz")
tables

In [ ]:
for table in tables:
    print(table)
    for batch in load_batches(table):
        db.insert_variants(batch)